In [ ]:
#Importing libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

from IPython.display import clear_output

#File names and folders
MinBiasdir = "minbiasdir06-05\\" # MB folder name
MinBiastrainfilename = "minbias_train.h5" # MB train test and validation name
MinBiastestfilename = "minbias_test.h5"
MinBiasvalidfilename = "minbias_valid.h5"
MinBiasvalidptestfilename = "minbias_valid_p_test.h5" # Valid and test set added together for larger set

LoneMuondir = "lonemuondir06-05\\" # Same for LM
LoneMuontrainfilename = "lone_muon_train_new.h5"
LoneMuontestfilename = "lone_muon_test_new.h5"
LoneMuonvalidfilename = "lone_muon_valid_new.h5"
LoneMuonvalidptestfilename = "lone_muon_valid_p_test_13-10_new.h5"


#Starting parameters
nSl = 6 # Number of slices
nL = 8

nbinsphi0 = 216+2*6 #phi-bins
nbinsQOpT = 216+2*2 #pT-bins

size_x = nbinsphi0
size_y = nbinsQOpT

In [ ]:
#Getting the files

#Train HDF5 file
MinBiastrainstorage = pd.HDFStore(MinBiasdir + MinBiastrainfilename)
LoneMuontrainstorage = pd.HDFStore(LoneMuondir + LoneMuontrainfilename)

#Train key list to file
Minbiastrainlist = MinBiastrainstorage.keys()
LoneMuontrainlist = LoneMuontrainstorage.keys()

#Length of lists
nMinbiastrainlist = len(Minbiastrainlist)
nLoneMuontrainlist = len(LoneMuontrainlist)




MinBiasvalidstorage = pd.HDFStore(MinBiasdir + MinBiasvalidfilename)
LoneMuonvalidstorage = pd.HDFStore(LoneMuondir + LoneMuonvalidfilename)

Minbiasvalidlist = MinBiasvalidstorage.keys()
LoneMuonvalidlist = LoneMuonvalidstorage.keys()

nMinbiasvalidlist = len(Minbiasvalidlist)
nLoneMuonvalidlist = len(LoneMuonvalidlist)




MinBiasteststorage = pd.HDFStore(MinBiasdir + MinBiastestfilename)
LoneMuonteststorage = pd.HDFStore(LoneMuondir + LoneMuontestfilename)

Minbiastestlist = MinBiasteststorage.keys()
LoneMuontestlist = LoneMuonteststorage.keys()

nMinbiastestlist = len(Minbiastestlist)
nLoneMuontestlist = len(LoneMuontestlist)




MinBiasvalidpteststorage = pd.HDFStore(MinBiasdir + MinBiasvalidptestfilename)
LoneMuonvalidpteststorage = pd.HDFStore(LoneMuondir + LoneMuonvalidptestfilename)

Minbiasvalidptestlist = MinBiasvalidpteststorage.keys()
LoneMuonvalidptestlist = LoneMuonvalidpteststorage.keys()

nMinbiasvalidptestlist = len(Minbiasvalidptestlist)
nLoneMuonvalidptestlist = len(LoneMuonvalidptestlist)



In [ ]:
def get_batch(storage, keys, i):
    batch = storage[keys[i]]
    return batch

In [ ]:
def get_images_allslices(df):
    n_events = int(len(df.index)/nSl/nL)
    
    images = torch.empty(0,nSl,nL,nbinsQOpT,nbinsphi0, dtype = torch.int)
    for i in range(n_events):
        slices = torch.empty(0,nL,nbinsQOpT,nbinsphi0, dtype = torch.int)
        for j in range(nSl):
            layers = torch.empty(0,nbinsQOpT,nbinsphi0, dtype = torch.int)
            for k in range(nL):
                layer = torch.zeros((nbinsQOpT,nbinsphi0), dtype = torch.int)
                layerx = df['x_pos'].iloc[i*nSl*nL + j*nL + k]
                layery = df['y_pos'].iloc[i*nSl*nL + j*nL + k]
                value = df['value'].iloc[i*nSl*nL + j*nL + k]
                layerxnp = ak.to_numpy(layerx).astype(int)
                layerynp = ak.to_numpy(layery).astype(int)
                valuenp = ak.to_numpy(value).astype(int)
                layer[layerynp,layerxnp] = torch.from_numpy(valuenp)

                layers = torch.cat((layers, layer.unsqueeze(0)),0)

            slices = torch.cat((slices, layers.unsqueeze(0)),0)
    
        images = torch.cat((images, slices.unsqueeze(0)),0)
                
    return images
                

In [ ]:
def get_max_center_2_all_slices(image):
    summedimage = torch.sum(image.gt(0),2)
    batchlist = []
    for ievent, event in enumerate(summedimage):
        imagemax = torch.max(event)
#         eventlist = []
#         print("max:",imagemax)
#         if imagemax >= 6:
        for islice, slice in enumerate(event):
            slicelist = []
            slicemax = torch.max(slice)
            if slicemax >= 6:
                for q in range(nbinsQOpT):
                    for phi in range(nbinsphi0):
                        if slice[q,phi] == slicemax:
                            slicelist += [[q,phi]]
#                             print("point added")
#                             print(eventlist)

            else:
                slicelist += []
#             print("empty")
#             print(eventlist)

            batchlist += [slicelist]
            
#         print(eventlist)
            
#     print(batchlist)
    newlist = []
    
    for islice, slice in enumerate(batchlist):
#         print(event)
        if len(slice)>0:
            torchslice = torch.tensor(slice)
            meanval = torch.mean(torchslice.float(),0)
            if (meanval[0] >= 1 - 0.5) and (meanval[0] <= (nbinsQOpT - 1) - 1 - 0.5) and (meanval[1] >= 2 - 0.5) and (meanval[1] <= (nbinsphi0 - 1) - 2 - 0.5):
                itrueevent = int(islice/nSl)
                itrueslice = islice - itrueevent*nSl
                slicemax = torch.max(summedimage[itrueevent, itrueslice])

                p0 = int(meanval[0]+1/2)
                p1 = int(meanval[1]+1/2)

    #             vicinity = torch.zeros(3,3)
    #             p1min = max(p1 - 2, 0)
    #             p1max = min(p1 + 3, nbinsphi0)
    #             p0min = max(p0 - 2, 0)
    #             p0max = min(p0 + 3, nbinsQOpT)
    #             vicinity[p0min - (p0 - 2):5 - (p0max - (p0 + 3)), p1min - (p1 - 2):5 - (p1max - (p1 + 3))] = summedimage[ievent, 0, p0min:p0max, p1min:p1max]

    #             vicp0min = vicinity

    #             p0lowerbound = 0 + 2 + 1
    #             p1lowerbound = 0 + 2 + 2
    #             p0upperbound = (nbinsQOpT - 1) - 2 - 1
    #             p1upperbound = (nbinsphi0 - 1) - 2 - 2
                p0lowerbound = 0 + 1
                p1lowerbound = 0 + 2
                p0upperbound = (nbinsQOpT - 1) - 1
                p1upperbound = (nbinsphi0 - 1) - 2

                p0min1 = max(p0 - 1, p0lowerbound)
                p1min1 = max(p1 - 2, p1lowerbound)
                p0max1 = min(p0 + 2, p0upperbound)
                p1max1 = min(p1 + 3, p1upperbound)

                vicinity1 = summedimage[itrueevent, itrueslice, p0min1:p0max1, p1min1:p1max1]
                sufficients = vicinity1.eq(slicemax)
                if torch.sum(sufficients) > 0:
                    
                
                    print(vicinity1)
#                     p1pltmin = max(p1 - 10, 0)
#                     p1pltmax = min(p1 + 10, nbinsphi0 - 1)
#                     p0pltmin = max(p0 - 10, 0)
#                     p0pltmax = min(p0 + 10, nbinsQOpT - 1)
#                     plt.imshow(summedimage[itrueevent, 0, p0pltmin:p0pltmax, p1pltmin:p1pltmax])
#                     plt.show()
#                     plt.imshow(summedimage[i, 0, p1 - 10:p1 + 10, p0 - 10:p0 + 10])

                    steepness_along = 2*vicinity1 - (summedimage[itrueevent, itrueslice, p0min1 - 1:p0max1 - 1, p1min1 - 2:p1max1 - 2]
                                                     + summedimage[itrueevent, itrueslice, p0min1 + 1:p0max1 + 1, p1min1 + 2:p1max1 + 2])

                    steepness_across = 2*vicinity1 - (summedimage[itrueevent, itrueslice, p0min1 - 1:p0max1 - 1, p1min1 + 1:p1max1 + 1]
                                                     + summedimage[itrueevent, itrueslice, p0min1 + 1:p0max1 + 1, p1min1 - 1:p1max1 - 1])


                    indices = torch.nonzero(sufficients)

                    score = - steepness_along + 0.01*steepness_across

                    best = torch.argmax(score[sufficients])

                    bestpoint = indices[best]

        #             if torch.max(summedimage[ievent]) == summedimage[ievent, 0, int(meanval[0]+1/2), int(meanval[1]+1/2)]: #Only implemented for 1 slice!!!!
                    
                    newlist += [[itrueevent, itrueslice, bestpoint[0] + p0min1, bestpoint[1] + p1min1]]
                
                else:
                    print(slicemax)
                    print(vicinity1)
        
    return slicelist, newlist

In [ ]:
#Notice that this should be done for every train test and validation file you wanna use.
path = LoneMuondir
newfile = "lone_muon_valid_p_test_13-10_new.h5" #insert own file names
newstorage = pd.HDFStore(LoneMuondir + newfile)
oldfile = "lone_muon_valid_p_test_13-10.h5" #insert own file names
old_storage = pd.HDFStore(LoneMuondir + oldfile) #Opens storages
old_list = old_storage.keys() #Gets keys
nlist = len(old_list) #Number of mini batches
for i in range(nlist):
    print(i,nlist)
    LMpd = get_batch(old_storage, old_list, i) #Load batch i
    LM = get_images_allslices(LMpd) #Get LM image
    something, LMtruth_1 = get_max_center_2_all_slices(LM) #Get truth
    LMpd2 = LMpd[LMpd.columns[0:3]].copy() #tabel copy
    nrows = len(LMpd2.index)
    LMpd2['truth_qOpT'] = -np.ones(nrows) #Adds truth collumn
    LMpd2['truth_phi0'] = -np.ones(nrows)
    
    for element in LMtruth_1:
        print(element)
        idx0 = LMpd2.index[0]
        LMpd2.at[idx0 + element[0]*nSl*nL + element[1]*nL,'truth_qOpT'] = element[2] #Adds truth
        LMpd2.at[idx0 + element[0]*nSl*nL + element[1]*nL,'truth_phi0'] = element[3]
        
    newstorage[old_list[i]] = LMpd2 #Saves mini batch
    
    